In [ ]:
# Import packages
from collections import Counter
import csv
import itertools
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import f1_score
import statistics as st
from tcn import TCN
import tensorflow
from tensorflow.keras import Sequential
from tensorflow.keras.callbacks import Callback
from tensorflow.keras.layers import Dense, Input, Embedding, LSTM, Conv1D, MaxPooling1D, Flatten
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

Data preparation

In [ ]:
import csv
user_sessions = []
current_session_id = None
current_session = []
# Read dataset
with open("browsing_train.csv") as csvfile:
    reader = csv.DictReader(csvfile)
    for idx, row in enumerate(reader):
            
        # Row will contain: session_id_hash, product_action, product_sku_hash
         _session_id_hash = row['session_id_hash']
         # When a new session begins, store the old one and start again
        if current_session_id and current_session and _session_id_hash != current_session_id:
            user_sessions.append(current_session)
            # Resets session
            current_session = []
        # We extract events from session
        if row['product_action'] == '' and row['event_type'] ==  'pageview':
            current_session.append('view')

        elif row['product_action'] != '':
            current_session.append(row['product_action'])
        # Update the current session id
        current_session_id = _session_id_hash

# Print how many sessions we have
print("# total sessions: {}".format(len(user_sessions)))
# Print first session
print("First session is: {}".format(user_sessions[0]))

In [ ]:
# Function to convert events to numbers and add start and stop token
def session_indexed(s):
    """
    Converts a session (of actions) to indices and adds start/end tokens
    :param s: list of actions in a session (i.e 'add','detail', etc)
    :return:
    """
    action_to_idx = {'start': 0, 'end': 1, 'add': 2, 'remove': 3, 'detail': 4, 'view': 5}
    return [action_to_idx['start']] + [action_to_idx[e] for e in s] + [action_to_idx['end']]

In [ ]:
purchase_sessions = []
abandon_sessions = []
browse_sessions = []
for s in user_sessions:
    # If add and purchase event in sessions and purchase event appears after add event...
    if 'purchase' in s and 'add' in s and s.index('purchase') > s.index('add'):
        p_session = s
        # Remove purchase event from session
        p_session = (p_session[:p_session.index("purchase")])
            
        # Remove clickstreams shorter than 5 or longer than 155 clicks
        if len(p_session) < 5 or len(p_session) > 155:
            continue
        else:
            # Append to list
            purchase_sessions.append(p_session)
        # Assert not any purchase event left in clickstream    
        assert not any( e == 'purchase' for e in p_session)

    # If add event and no purchase event in session...
    elif 'add' in s and not 'purchase' in s:
        if len(s) < 5 or len(s) > 155:
            continue
        else:
            abandon_sessions.append(s)
    # If no purchase event in session...    
    elif 'purchase' not in s:
        if len(s) < 5 or len(s) > 155:
            continue
        else:    
            browse_sessions.append(s)

In [ ]:
# Add start stop token, convert to numbers
purchase_sessions = [session_indexed(s) for s in purchase_sessions]
abandon_sessions = [session_indexed(s) for s in abandon_sessions]
browse_sessions = [session_indexed(s) for s in browse_sessions]

# Combine sessions into final dataset
x = purchase_sessions + abandon_sessions + browse_sessions

# give label=1 for purchase, label=0 for abandon, label=2 for browse
y = [1]*len(purchase_sessions) +[0]*len(abandon_sessions) + [2]*len(browse_sessions)
assert len(x) == len(y)

In [ ]:
print("# total sessions after data prep: {}".format(len(x)))
percentage_left = 100 - (len(x)/len(user_sessions) *100)
print("% total sessions left after data prep: {}".format(round(len(x)/len(user_sessions)*100),2))
print("% drop: {}".format(round(percentage_left),2))
print("\n")
print("Remaining dataset:")
print("%abandon sessions: {}".format(round(((len(abandon_sessions)/len(x))*100),2)))
print("%purchase sessions: {}".format(round(((len(purchase_sessions)/len(x))*100),2)))
print("%browse sessions: {}".format(round(((len(browse_sessions)/len(x))*100),2)))

Split data into train, val and test set

In [ ]:
from sklearn.model_selection import train_test_split
# First, split the data in training and remaining dataset
X_train, X_rem, y_train, y_rem = train_test_split(x,y, train_size=0.7, stratify = y, random_state = 3340)

In [ ]:
# Second, split the remaining data into a validation and test set
test_size = 0.5
X_valid, X_test, y_valid, y_test = train_test_split(X_rem,y_rem, test_size=0.5, stratify = y_rem, random_state = 3340)

In [ ]:
print("Size of training data: {}, validation data: {}, test data: {}".format(len(X_train), len(X_valid), len(X_test)))

Exploring data

In [ ]:
# Convert training set to seperate sets per class
def converttosessions(X_train, y_train):
    tupletrainitems = [tuple(x) for x in X_train]
    tuplex = tuple(tupletrainitems)
    tupley = tuple(y_train)
    newdic = zip(tupley,tuplex)
    
    abandon_sessions = []
    purchase_sessions = []
    browsing_sessions = []
    
    for x in list(newdic):
        if x[0] == 0:
            a = list(x[1])
            b = a[1:-1]
            abandon_sessions.append(b)
        elif x[0] == 1:
            a = list(x[1])
            b = a[1:-1]
            purchase_sessions.append(b)
        elif x[0] == 2:
            a = list(x[1])
            b = a[1:-1]
            browsing_sessions.append(b)
    
    return abandon_sessions, purchase_sessions, browsing_sessions

In [ ]:
train_abandon_sessions, train_purchase_sessions, train_browsing_sessions = converttosessions(X_train, y_train)

In [ ]:
# Some statistics on the training dataset
def statistics(X_train, abandon_sessions, purchase_sessions, browsing_sessions):
    length_abandon = len(abandon_sessions)
    length_purchase = len(purchase_sessions)
    length_browsing = len(browsing_sessions)
    print("Number of clickstreams per category, abandon: {}, purchase: {}, browsing: {}".format(length_abandon, length_purchase, length_browsing))
    
    session_lengths = []
    abandon_lengths = []
    purchase_lengths = []
    browsing_lengths = []
    for x in X_train:
        session_lengths.append(len(x)-2)
    for x in abandon_sessions:
        abandon_lengths.append(len(x))
    for x in purchase_sessions:
        purchase_lengths.append(len(x))
    for x in browsing_sessions:
        browsing_lengths.append(len(x))
    
    length_total = len(session_lengths)
    print("Total number of clickstreams: {}".format(length_total))
    perc_abandon = round(((length_abandon/length_total) * 100),2)
    perc_purchase = round(((length_purchase/length_total) * 100),2)
    perc_browsing = round(((length_browsing/length_total) * 100),2)
    print("Percentage of clickstreams per category, abandon: {}, purchase: {}, browsing: {}".format(perc_abandon, perc_purchase, perc_browsing))
    
    
    shortest_session = min(session_lengths)
    longest_session = max(session_lengths)
    average_session = round(st.mean(session_lengths))
    median_session = int(st.median(session_lengths))
    std_session = round(st.stdev(session_lengths))
    
    print("Shortest total session length: {}, longest total session length: {}".format(shortest_session, longest_session))
    print("Average total session length: {}, standard deviation: {}".format(average_session, std_session))
    print("Median total session length: {}".format(median_session))
    
    shortest_abandon = min(abandon_lengths)
    shortest_purchase = min(purchase_lengths)
    shortest_browsing = min(browsing_lengths)
    print("Shortest clickstream per category, abandon: {}, purchase: {}, browsing: {}".format(shortest_abandon, shortest_purchase, shortest_browsing))
          
    longest_abandon = max(abandon_lengths)
    longest_purchase = max(purchase_lengths)
    longest_browsing = max(browsing_lengths)
    print("Longest clickstream per category, abandon: {}, purchase: {}, browsing: {}".format(longest_abandon, longest_purchase, longest_browsing))

    average_total = round(st.mean(session_lengths), 2)
    average_abandon = round(st.mean(abandon_lengths), 2)
    average_purchase = round(st.mean(purchase_lengths), 2)
    average_browsing = round(st.mean(browsing_lengths), 2)
    print("Average clickstream length all sessions: {}".format(average_total))
    print("Average clickstream length per category, abandon: {}, purchase: {}, browsing: {}".format(average_abandon, average_purchase, average_browsing))
    
    std_total = round(st.stdev(session_lengths), 2)
    std_abandon = round(st.stdev(abandon_lengths), 2)
    std_purchase = round(st.stdev(purchase_lengths), 2)
    std_browsing = round(st.stdev(browsing_lengths), 2)
    print("Standard deviation of mean total clickstreams: {}".format(std_total))       
    print("Standard deviation of mean clickstream per category, abandon: {}, purchase: {}, browsing: {}".format(std_abandon, std_purchase, std_browsing))

In [ ]:
statistics(X_train, train_abandon_sessions, train_purchase_sessions, train_browsing_sessions)

In [ ]:
# Visualising three clickstreams
def visualizeclickstream(abandon_sessions, purchase_sessions, browsing_sessions):
    abandon_clickstream = 0
    purchase_clickstream = 0
    browsing_clickstream = 0
    length = list(range(1,20+1))
    
    for x in abandon_sessions:
        if len(x) == length[-1] and 3 in x:
            abandon_clickstream = x
    for x in purchase_sessions:
        if len(x) == length[-1] and 3 in x:
            purchase_clickstream = x
    for x in browsing_sessions:
        if len(x) == length[-1] and 3 in x:
            browsing_clickstream = x

    # Plot a simple line chart
    plt.figure(figsize=(8,4))
    plt.plot(length, abandon_clickstream, marker='s')

    # Plot another line on the same chart/graph
    plt.plot(length, purchase_clickstream, marker='p', linestyle='--')
    
    plt.plot(length, browsing_clickstream, marker='o', linestyle=':')

    
    #{add': 2, 'remove': 3, 'detail': 4, 'view': 5}
    y = [2,3,4,5]
    yticks = ['Add', 'Remove', 'Detail', 'View']
    xticks = list(range(0,21, 2))
    plt.yticks(y, yticks)
    plt.xticks(xticks)
    
    #Invert y-axis for legible plot
    plt.gca().invert_yaxis()
    plt.legend(['Abandon', 'Purchase', 'Browsing-Only'])
    plt.xlabel("Number of Clicks")
    plt.ylabel("Event Type")
    plt.show()

In [ ]:
visualizeclickstream(train_abandon_sessions, train_purchase_sessions, train_browsing_sessions)

In [ ]:
# Visualising events per class
def eventsperclass(abandon_sessions, purchase_sessions, browsing_sessions):
    abandoncounter = Counter(itertools.chain(*abandon_sessions))
    abandoncounter = dict(sorted(abandoncounter.items(), key=lambda item: item[0]))
    purchasecounter = Counter(itertools.chain(*purchase_sessions))
    purchasecounter = dict(sorted(purchasecounter.items(), key=lambda item: item[0]))
    browsingcounter = Counter(itertools.chain(*browsing_sessions))
    browsingcounter = dict(sorted(browsingcounter.items(), key=lambda item: item[0]))
    
    abandon_list = list(abandoncounter.values())
    purchase_list = list(purchasecounter.values())
    browsing_list = list(browsingcounter.values())
    
    print("Total number of events per class:")
    print("Abandon: {}, purchase: {}, browsing: {}".format(abandoncounter, purchasecounter, browsingcounter))
    print("\n")
    length_abandon = sum(abandon_list)
    length_purchase = sum(purchase_list)
    length_browsing = sum(browsing_list)
    
    #{add': 2, 'remove': 3, 'detail': 4, 'view': 5}
    perc_addabandon = round(((abandon_list[0]/length_abandon) * 100),2)
    perc_addpurchase = round(((purchase_list[0]/length_purchase) * 100),2)
    print("Percentage of add events in abandon: {}, purchase: {}".format(perc_addabandon, perc_addpurchase))
    
    perc_removeabandon = round(((abandon_list[1]/length_abandon) * 100),2)
    perc_removepurchase = round(((purchase_list[1]/length_purchase) * 100),2)
    perc_removebrowsing = round(((browsing_list[0]/length_browsing) * 100),2)
    print("Percentage of remove events in abandon: {}, purchase: {}, browsing: {}".format(perc_removeabandon, perc_removepurchase, perc_removebrowsing))
    
    perc_detailabandon = round(((abandon_list[2]/length_abandon) * 100),2)
    perc_detailpurchase = round(((purchase_list[2]/length_purchase) * 100),2)
    perc_detailbrowsing = round(((browsing_list[1]/length_browsing) * 100),2)
    print("Percentage of detail events in abandon: {}, purchase: {}, browsing: {}".format(perc_detailabandon, perc_detailpurchase, perc_detailbrowsing))
    
    perc_viewabandon = round(((abandon_list[3]/length_abandon) * 100),2)
    perc_viewpurchase = round(((purchase_list[3]/length_purchase) * 100),2)
    perc_viewbrowsing= round(((browsing_list[2]/length_browsing) * 100),2)
    print("Percentage of view events in abandon: {}, purchase: {}, browsing: {}".format(perc_viewabandon, perc_viewpurchase, perc_viewbrowsing))
    
    add_percentages = [perc_addabandon, perc_addpurchase, 0]
    remove_percentages = [perc_removeabandon, perc_removepurchase, perc_removebrowsing]
    detail_percentages = [perc_detailabandon, perc_detailpurchase, perc_detailbrowsing]
    view_percentages = [perc_viewabandon, perc_viewpurchase, perc_viewbrowsing]
    %matplotlib inline
    df = pd.DataFrame({'Class': ["Abandon", "Purchase", "Browsing-Only"], 'Add': add_percentages, 'Remove': remove_percentages,
                      'Detail': detail_percentages, 'View': view_percentages})
    print(df)
    df.plot(x='Class', kind='bar', stacked=True)
    plt.legend(bbox_to_anchor=(1.0, 1.0))
    plt.xticks(rotation=0)
    plt.ylabel("Percentage")
    plt.show()

In [ ]:
eventsperclass(train_abandon_sessions, train_purchase_sessions, train_browsing_sessions)

Padding & one-hot encoding

In [ ]:
# Pad sequences
max_len = 157
X_train = pad_sequences(X_train, padding="post",value=6, maxlen=max_len)
X_valid = pad_sequences(X_valid, padding="post", value=6, maxlen=max_len)
X_test = pad_sequences(X_test, padding="post", value=6, maxlen=max_len)

In [ ]:
# Convert to one-hot
X_train = tf.one_hot(X_train, depth=7)
X_valid = tf.one_hot(X_valid, depth=7)
X_test = tf.one_hot(X_test, depth=7)

In [ ]:
# Convert labels to arrays
y_train = np.array(y_train)
y_valid = np.array(y_valid)

#One-hot encode labels
y_train = to_categorical(y_train, 3)
y_valid = to_categorical(y_valid, 3)

# Save for metric calculations
test_labels = y_test

Gridsearch

CNN

In [ ]:
def cnngridsearch(filter_size, neurons, kernel_size):
    batch = 32
    epochs = 20
    patience = 5
    l = 0.001
    #Hyperparameters
    opt = keras.optimizers.Adam(l)
    loss = keras.losses.CategoricalCrossentropy()
    es = keras.callbacks.EarlyStopping(monitor='loss',
                                       patience=patience,
                                       verbose=1,
                                       restore_best_weights=True)
    
    model = Sequential()
    # Input layer
    model.add(Input(shape = (X_train.shape[1], X_train.shape[2])))
    # Convolutional layer
    model.add(Conv1D(filter_size, kernel_size, activation='relu'))
    # Pooling layer
    model.add(MaxPooling1D(pool_size = 2))    
    # Flatten layer
    model.add(Flatten())
    # Fully connected layer
    model.add(Dense(neurons, activation='relu'))
    # Output layer
    model.add(Dense(y_train.shape[1],activation='softmax'))
    
    model.compile(optimizer=opt,
                loss=loss,
                metrics=['categorical_accuracy'])
    
    model.fit(X_train, y_train,
                    epochs = epochs,
                    batch_size = batch,
                    callbacks = es)
    
    return model

In [ ]:
# Hyperparameters to be tested
neurons = [32, 64, 128]
kernel_num = [3, 5, 7]
filter_num = [32, 64, 128]
f1_scores = dict()

for n in neurons:
    print("Testing neurons:", n)
    for k in kernel_num:
        print("Testing kernel size:", k)
        for f in filter_num:
            print("Testing filters", f)
            print("Fitting model")
            cnn = cnngridsearch(f, n, k)
            
            #Calculating y_pred
            y_pred_validate = cnn.predict(X_valid)
            rounded = np.argmax(np.round(y_pred_validate),axis=1)
            rounded = list(rounded)
        
            #Evaluating the model
            f1score = f1_score(valid_labels, rounded, average = "macro")
            print("Macro-averaged F1 score:", f1score)
        
            #Appending evaluations to dictionaries
            f1_scores[f1score] = (f, n, k)

In [ ]:
#Sort F1 scores from highest to lowest
sortscores = {key: val for key, val in sorted(f1_scores.items(), key = lambda ele: ele[0])}
print("Result dictionary sorted by F1 score : " + str(sortscores))

TCN

In [ ]:
def tcngridsearch(filter_size, kernel_size, dilations):
    epochs=20
    patience=5
    batch=32
    l=0.001
    
    #Hyperparameters
    opt = keras.optimizers.Adam(l)
    loss = keras.losses.CategoricalCrossentropy()
    es = keras.callbacks.EarlyStopping(monitor='loss',
                                       patience=patience,
                                       verbose=1,
                                       restore_best_weights=True)

    # Define Model
    model = keras.Sequential()
    # Input layer
    model.add(Input(shape = (X_train.shape[1], X_train.shape[2])))
    # TCN layer
    model.add(TCN(
        nb_filters= filter_size,
        kernel_size=kernel_size,
        dilations=dilations
        ))
    # Output layer
    model.add(Dense(y_train.shape[1],activation='softmax'))

    model.compile(optimizer=opt,
                loss=loss,
                metrics=['categorical_accuracy'])
    
    model.fit(X_train, y_train,
                    epochs = epochs,
                    batch_size = batch,
                    callbacks = es)
    
    return model

In [ ]:
kernel_num = [3, 5, 7]
filter_num = [32, 64, 128]
dilations = [[1, 2, 4, 8, 16, 32, 64], [1, 2, 4, 8, 16, 32]]
f1_scores = dict()

# Calculate receptive field
print("Receptive fields")
print("For kernel size 3: {}".format(3*1*64))
print("For kernel size 5: {}".format(5*1*32))
print("For kernel size 7: {}".format(7*1*32))

In [ ]:
for f in filter_num:
    print("Testing filter size:", f)
    for k in kernel_num:
        print("Testing kernel size:", k)
        if k == 3:
            d = dilations[0]
            print("Last dilation:", d[-1])
        if k == 5 or k == 7:
            d = dilations[1]
            print("Last dilation:", d[-1])
        
        print("Fitting model")
        tcn = tcngridsearch(f, k, d)
            
        #Calculating y_pred
        y_pred_validate = tcn.predict(X_valid)
        rounded = np.argmax(np.round(y_pred_validate),axis=1)
        rounded = list(rounded)
        
        #Evaluating the model
        f1score = f1_score(valid_labels, rounded, average = "macro")
        print("f1 score for this model:", f1score)
        
        #Appending evaluations to dictionaries
        f1_scores[f1score] = (f, k, d)

In [ ]:
#Sort F1 scores from highest to lowest
sortscores = {key: val for key, val in sorted(f1_scores.items(), key = lambda ele: ele[0])}
print("Result dictionary sorted by F1 score : " + str(sortscores))

Testing models

LSTM

In [ ]:
# Calculate F1 scores and configure confusion matrices
def metric_calculation(predictions):
    rounded = np.argmax(np.round(predictions),axis=1)
    rounded = list(rounded)
    f1 = f1_score(test_labels, rounded, average='macro')
    print ("f1macro: {}".format(round(f1, 3)))
    print(metrics.classification_report(test_labels, rounded, digits=3))
    
    target_names = ["Abandon", "Purchase", "Browsing-only"]
    cm = metrics.confusion_matrix(test_labels, rounded)
    cmn = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    heatmap = sns.heatmap(cmn, annot=True, fmt='.2f', xticklabels=target_names, yticklabels=target_names, cmap=plt.cm.Blues)
    plt.ylabel('Actual class')
    plt.xlabel('Predicted class')
    figure = heatmap.get_figure()    
    figure.savefig('cm.png')
    
    return heatmap

In [ ]:
# Baseline
def lstmmodel(X_train, y_train, X_val, y_val):
    #Hyperparamaters
    lr = 0.001
    batch = 32
    epochs = 50
    patience = 10
    opt = keras.optimizers.Adam(learning_rate=lr)
    loss = keras.losses.CategoricalCrossentropy()
    es = keras.callbacks.EarlyStopping(monitor='val_loss',
                                       patience=patience,
                                       verbose=1,
                                       restore_best_weights=True)
    
    model = Sequential()
    # Input layer
    model.add(Input(shape = (X_train.shape[1], X_train.shape[2])))
    # LSTM layer
    model.add(LSTM(64)) 
    # Output layer
    model.add(Dense(y_train.shape[1],activation='softmax'))
    model.summary()
    
    model.compile(optimizer=opt,
                loss=loss,
                metrics=['categorical_accuracy'])
    
    model.fit(X_train, y_train,
                    validation_data=(X_val, y_val),
                    epochs = epochs,
                    batch_size = batch,
                    callbacks = es)
    
    return model

In [ ]:
def lstmpredictions(X_test):
    lstm = lstmmodel(X_train, y_train, X_valid, y_valid)
    preds = lstm.predict(X_test,batch_size=32)
    return preds

In [ ]:
lstm_preds = lstmpredictions(X_test)

In [ ]:
metric_calculation(lstm_preds)

CNN

In [ ]:
def cnnmodel(X_train, y_train, X_val, y_val):
    lr = 0.001
    batch = 32
    epochs = 50
    patience = 10
    opt = keras.optimizers.Adam(learning_rate = lr)
    loss = keras.losses.CategoricalCrossentropy()
    es = keras.callbacks.EarlyStopping(monitor='val_loss',
                                       patience=patience,
                                       verbose=1,
                                       restore_best_weights=True)
    
    model = Sequential()
    # Input layer
    model.add(Input(shape = (X_train.shape[1], X_train.shape[2])))
    # Convolutional layer
    model.add(Conv1D(filters = 32, kernel_size = 7, activation='relu'))
    # Pooling layer
    model.add(MaxPooling1D(pool_size = 2))    
    # Flatten layer
    model.add(Flatten())
    # Fully connected layer
    model.add(Dense(64, activation='relu'))
    # Output layer
    model.add(Dense(y_train.shape[1],activation='softmax'))
    model.summary()
    
    model.compile(optimizer=opt,
                loss=loss,
                metrics=['categorical_accuracy'])
    
    model.fit(X_train, y_train,
                    validation_data=(X_val, y_val),
                    epochs = epochs,
                    batch_size = batch,
                    callbacks = es)
    
    return model

In [ ]:
def cnnpredictions(X_test):
    cnn = cnnmodel(X_train, y_train, X_valid, y_valid)
    preds = cnn.predict(X_test,batch_size=32)
    return preds

In [ ]:
cnn_preds = cnnpredictions(X_test)

In [ ]:
metric_calculation(cnn_preds)

CNN-LSTM

In [ ]:
def cnnlstmmodel(X_train, y_train, X_val, y_val):
    lr = 0.001
    batch = 32
    epochs = 50
    patience = 10
    opt = keras.optimizers.Adam(learning_rate = lr)
    loss = keras.losses.CategoricalCrossentropy()
    es = keras.callbacks.EarlyStopping(monitor='val_loss',
                                       patience=patience,
                                       verbose=1,
                                       restore_best_weights=True)
    
    model = Sequential()
    # Input layer
    model.add(Input(shape = (X_train.shape[1], X_train.shape[2])))
    # Convolutional layer
    model.add(Conv1D(filters = 32, kernel_size = 7, activation='relu'))
    # Pooling layer
    model.add(MaxPooling1D(pool_size = 2))  
    #LSTM layer
    model.add(LSTM(64))
    # Output layer
    model.add(Dense(y_train.shape[1],activation='softmax'))
    model.summary()
    
    model.compile(optimizer=opt,
                loss=loss,
                metrics=['categorical_accuracy'])
    
    model.fit(X_train, y_train,
                    validation_data=(X_val, y_val),
                    epochs = epochs,
                    batch_size = batch,
                    callbacks = es)
    
    return model

In [ ]:
def cnnlstmpredictions(X_test):
    cnnlstm = cnnlstmmodel(X_train, y_train, X_valid, y_valid)
    preds = cnnlstm.predict(X_test,batch_size=32)
    return preds

In [ ]:
cnnlstm_preds = cnnlstmpredictions(X_test)

In [ ]:
metric_calculation(cnnlstm_preds)

TCN

In [ ]:
def tcnmodel(X_train, y_train, X_val, y_val):
    epochs=50
    patience=10 
    batch=32
    lr=0.001 
    opt = keras.optimizers.Adam(learning_rate = lr)
    loss = keras.losses.CategoricalCrossentropy()
    es = keras.callbacks.EarlyStopping(monitor='val_loss',
                                       patience=patience,
                                       verbose=1,
                                       restore_best_weights=True)

    # Define Model
    model = keras.Sequential()
    model.add(Input(shape = (X_train.shape[1], X_train.shape[2])))
    model.add(TCN(
        nb_filters=64,
        kernel_size=7,
        dilations=[1, 2, 4, 8, 16, 32]
        ))
    model.add(keras.layers.Dense(y_train.shape[1], activation='softmax'))
    model.summary()

    model.compile(optimizer=opt,
                loss=loss,
                metrics=['categorical_accuracy'])
    
    model.fit(X_train, y_train,
                    validation_data=(X_val, y_val),
                    epochs = epochs,
                    batch_size = batch,
                    callbacks = es)
    return model

In [ ]:
def tcnpredictions(X_test):
    tcn = tcnmodel(X_train, y_train, X_valid, y_valid)
    preds = tcn.predict(X_test,batch_size=32)
    return preds

In [ ]:
tcn_preds = tcnpredictions(X_test)

In [ ]:
metric_calculation(tcn_preds)